In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
base = "../"
#base = "../victims/sphincsplus/ref/"
keys_file = base + "keys.txt"
sigs_file = base + "sigs.txt"

In [3]:
def print_adrs(adrs, end='\n'):
    hex = adrs.hex()
    print(' '.join([hex[i:i+8] for i in range(0, len(hex), 8)]), end=' ')
    print(end=end)

In [4]:
#   A class for handling Addresses (Section 4.2.)
class ADRS:
    #   type constants
    WOTS_HASH   = 0
    WOTS_PK     = 1
    TREE        = 2
    FORS_TREE   = 3
    FORS_ROOTS  = 4
    WOTS_PRF    = 5
    FORS_PRF    = 6

    def __init__(self, a=32):
        """Initialize."""
        self.a = bytearray(a)

    def copy(self):
        """ Make a copy of self."""
        return ADRS(self.a)

    def set_layer_address(self, x):
        """ Set layer address."""
        self.a[ 0: 4] = x.to_bytes(4, byteorder='big')

    def set_tree_address(self, x):
        """ Set tree address."""
        self.a[ 4:16] = x.to_bytes(12, byteorder='big')

    def set_key_pair_address(self, x):
        """ Set key pair Address."""
        self.a[20:24] = x.to_bytes(4, byteorder='big')

    def get_key_pair_address(self):
        """ Get key pair Address."""
        return int.from_bytes(self.a[20:24], byteorder='big')

    def set_tree_height(self, x):
        """ Set FORS tree height."""
        self.a[24:28] = x.to_bytes(4, byteorder='big')

    def set_chain_address(self, x):
        """ Set WOTS+ chain address."""
        self.a[24:28] = x.to_bytes(4, byteorder='big')

    def set_tree_index(self, x):
        """ Set FORS tree index."""
        self.a[28:32] = x.to_bytes(4, byteorder='big')

    def get_tree_index(self):
        """ Get FORS tree index."""
        return int.from_bytes(self.a[28:32], byteorder='big')

    def set_hash_address(self, x):
        """ Set WOTS+ hash address."""
        self.a[28:32] = x.to_bytes(4, byteorder='big')

    def set_type_and_clear(self, t):
        """ The member function ADRS.setTypeAndClear(Y) for addresses sets
            the type of the ADRS to Y and sets the fnal 12 bytes of the ADRS
            to zero."""
        self.a[16:20] = t.to_bytes(4, byteorder='big')
        for i in range(12):
            self.a[20 + i] = 0

    def adrs(self):
        """ Return the ADRS as bytes."""
        return self.a

    def adrsc(self):
        """ Compressed address ADRDc used with SHA-2."""
        return self.a[3:4] + self.a[8 : 16] + self.a[19:20] + self.a[20:32]


In [5]:
with open(sigs_file, "r") as f:
    sigs = [bytes.fromhex(s.strip()) for s in f.readlines()]
    #sigs = [(s[0], bytes.fromhex(s[1])) for s in sigs]
    #sigs_correct = [s[1] for s in sigs if s[0] == '[CORRECT]']
    #sigs_faulty = [s[1] for s in sigs if s[0] == '[FAULTY]']
print(f"Loaded {len(sigs)} signatures")

Loaded 245 signatures


In [ ]:
import fips205
def group_keys_by_addr(self: fips205.SLH_DSA, pk: bytes, sigs: list[bytes]) -> dict[fips205.ADRS, fips205.WOTSKeyData]:
    # we assume SPHINCS-SHAKE-256s
    a = self.a
    d = self.d
    hp = self.hp
    n = self.n
    k = self.k
    wots_len = self.len
    wots_bytes = wots_len * n
    xmss_bytes = hp * n
    fors_bytes = k*(n+a*n) # 10560
    print("WOTS bytes:", wots_bytes)
    print("XMSS bytes:", xmss_bytes)
    print("FORS bytes:", fors_bytes)
    sig_len = n + fors_bytes + d * (wots_bytes + xmss_bytes)
    # sig_fields = [('R', n), ('FORS', fors_bytes)] + [[(f'WOTS{i}', wots_bytes), (f'XMSS{i}', xmss_bytes)] for i in range(8)] + [('msg', 0)]
    
    pk_seed = pk[:n]
    pk_root = pk[n:]
    print("pk_seed:", pk_seed.hex())
    print("pk_root:", pk_root.hex())
    for sig in sigs:
        m = sig[sig_len:]
        sig = sig[:sig_len]
        wots_keys_before = self.wots_keys
        print(wots_keys_before)
        valid = self.slh_verify_internal(m, sig, pk)
        wots_keys_after = self.wots_keys
        print(wots_keys_after)
        for key, value in wots_keys_after.items():
            if key not in wots_keys_before or wots_keys_before[key] != value:
                print("Key changed:", key, value)
    return self.wots_keys

In [7]:
with open(keys_file, "r") as f:
    lines = [s.split(': ') for s in f.readlines()]
    keys = {s[0]: bytes.fromhex(s[1].strip()) for s in lines}
keys

{'pk': b"\x91'\x92\xdaU\xb3\x8b\x92\xecZ\xb9\x9bgRb\xcf\xefT\xe1\xcd\xceC\n\xf1\x94\xfa\xf8f\x9c7\xe5\xdcRrLE\x1c\x86$UJ'\xb3\xff9P\xd1&\xfbEa\xc2}i\x82\xa8_\xa8\x92\x18\xffQ\x93\x13",
 'sk': b"\x93\xf5\xc2%\\X\xde\xe9\x9a\x8an\xc3\x10\xf9\xc9\xe4lJ\xc7\xecr\x90\x80\xdc\xb7#\x14f\xfa|\x15\xcd\xf3pW\xc2CUo\xad\xe4\xd8\xde\x89E!\xad\xd3\xa0\xb1\xe6>K\xe7\x17\x17;\xb5\x8d\x86\xb1_K\x85\x91'\x92\xdaU\xb3\x8b\x92\xecZ\xb9\x9bgRb\xcf\xefT\xe1\xcd\xceC\n\xf1\x94\xfa\xf8f\x9c7\xe5\xdcRrLE\x1c\x86$UJ'\xb3\xff9P\xd1&\xfbEa\xc2}i\x82\xa8_\xa8\x92\x18\xffQ\x93\x13"}

In [8]:
slh_dsa = fips205.SLH_DSA('SLH-DSA-SHAKE-256s')
pk = keys['pk']
pk_seed = pk[:slh_dsa.n]
pk_root = pk[slh_dsa.n:]
groups = group_keys_by_addr(slh_dsa, keys['pk'], sigs)

groups = sorted(groups.items(), key=lambda item: len(item[1]), reverse=True)
for adrs, value in groups:
    # print key.hex(), add whitespace every 4 bytes
    print_adrs(adrs.adrs(), end='')
    print(len(value))
    pks = []
    for v in value:
        wots_pk = slh_dsa.wots_pk_from_sig(v.sig, v.msg, pk_seed, adrs)
        print(wots_pk.hex())

WOTS bytes: 2144
XMSS bytes: 256
FORS bytes: 10560
pk_seed: 912792da55b38b92ec5ab99b675262cfef54e1cdce430af194faf8669c37e5dc
pk_root: 52724c451c8624554a27b3ff3950d126fb4561c27d6982a85fa89218ff519313


00000000 00000000 0004b6cb 824912e4 00000001 000000a8 00000000 00000000 1
2707b665dadb503f8e5e3a8b723ae678e652165e5f10a424f742cfa65ff7ce00
00000001 00000000 000004b6 cb824912 00000001 000000e4 00000000 00000000 1
f8ae8ac1858b849bae552bc8fa65941701c59bad8082c1684dcdd721bd078987
00000002 00000000 00000004 b6cb8249 00000001 00000012 00000000 00000000 1
eebee79c391c77bee239cedd396d6c767c158c0657ae48e707bbae96d9614786
00000003 00000000 00000000 04b6cb82 00000001 00000049 00000000 00000000 1
26239ff3f59eb23881242024aacf4abcebf534a3c920660fe383557656d9cf43
00000004 00000000 00000000 0004b6cb 00000001 00000082 00000000 00000000 1
2df79c28ae58747031fd9e6d6f9bd3cd8ad7bc18638bb6e5500128bdff5e3b66
00000005 00000000 00000000 000004b6 00000001 000000cb 00000000 00000000 1
58dc00fe61f75044f2a54e2bb4bd4d1bcad152254373b85078c2f35b02dad436
00000006 00000000 00000000 00000004 00000001 000000b6 00000000 00000000 1
7c7a160151fb2d59ae1c489f93eca81cc6a1222f73643ae71f1acfe5c2d7fa1c
00000007 00000000 00000000 

In [9]:
import fips205 as sp
import postprocessing as pp

inst = 'SLH-DSA-SHAKE-256s'
instance = sp.SLH_DSA_PARAM[inst]

n = instance[1]

#keys = pp.read_keys("../victims/sphincsplus/ref/keys.txt")
#sk = keys["sk"]
#pk = keys["pk"]
#sk_seed = sk[:n]
#sk_prf = sk[n:2*n]
#pk_seed = pk[:n]
#pk_root = pk[n:]
sk_seed = bytes.fromhex("e17e72290e49a44c9c534f211195257cf13b0d45405782ceda2d7f982a551721")
sk_prf = bytes.fromhex("b47cfcf1b7764296d81055df05ff8295e4641ad9aa2db29b7b678e788bb8ea62")
pk_seed = bytes.fromhex("0847487e02a874ef8feee587f5359dfcd722f10e1cb50ac538d74320a5bfd242")
pk_root = bytes.fromhex("a9375b1b113255ce32578768690cd17431de27c356de7c7f34c057d81327b746")
print("sk_seed", sk_seed.hex())
print("sk_prf", sk_prf.hex())
print("pk_seed", pk_seed.hex())
print("pk_root", pk_root.hex())
slh_dsa = sp.SLH_DSA(inst)
(pk, sk) = slh_dsa.slh_keygen_internal(
    sk_seed=sk_seed,
    sk_prf=sk_prf,
    pk_seed=pk_seed,
    param=inst)
print("keygen done")
print("sk", sk.hex())
print("pk", pk.hex())

sk_seed e17e72290e49a44c9c534f211195257cf13b0d45405782ceda2d7f982a551721
sk_prf b47cfcf1b7764296d81055df05ff8295e4641ad9aa2db29b7b678e788bb8ea62
pk_seed 0847487e02a874ef8feee587f5359dfcd722f10e1cb50ac538d74320a5bfd242
pk_root a9375b1b113255ce32578768690cd17431de27c356de7c7f34c057d81327b746
keygen done
sk e17e72290e49a44c9c534f211195257cf13b0d45405782ceda2d7f982a551721b47cfcf1b7764296d81055df05ff8295e4641ad9aa2db29b7b678e788bb8ea620847487e02a874ef8feee587f5359dfcd722f10e1cb50ac538d74320a5bfd242903e394a806d960aa74d9f6c38ab84fb1f6bb7cf17f54423da5432d095d6444d
pk 0847487e02a874ef8feee587f5359dfcd722f10e1cb50ac538d74320a5bfd242903e394a806d960aa74d9f6c38ab84fb1f6bb7cf17f54423da5432d095d6444d
